### Tracking SSP Dynamics

This is a demo showing how SSP representations can be used to track the dynamics of a "ground-truth" environment defined using the CoppeliaSim physics simulator. To run this notebook in tandem with CoppeliaSim, all you need to do is have the scene `blocks.ttt` (found in the SSP library) open in the background.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np

from IPython.display import HTML
from ssp.interface import CoppeliaSim
from ssp.maps import Spatial2D
from ssp.plots import heatmap_animation, create_gif

In [ ]:
dim = 256
scale = 15
sim_steps = 50

# lengths matched to scene in CoppeliaSim
x_len = 2
y_len = 2
x_spaces = 101
y_spaces = 101

In [ ]:
objects = ['A', 'B']

ssp_map = Spatial2D(dim=dim, scale=scale)
ssp_map.build_grid(x_len, y_len, x_spaces, y_spaces)

heatmaps = []

# create the interface and heatmaps tracking sim state
with CoppeliaSim(dt=0.005) as interface:
    interface.set_sensor('Vision_sensor')
    
    for obj in objects:
        interface.create_object(obj)
        interface.apply_force(obj)
    
    for step in range(sim_steps):
        ssp_map.update_from_sim(interface)
        heatmaps.append(ssp_map.heatmap_scores)
        interface.advance()

In [ ]:
# plot CoppeliaSim and SSP representations for comparison
n_plots = 2
n_steps = len(interface.sensor_data)
figsize = (12, 6)
sims = [interface.sensor_data, heatmaps]

ani = heatmap_animation(sims, figsize=figsize)
HTML('<img src="data:image/gif;base64,{0}" />'.format(create_gif(ani)))